**Table of contents**<a id='toc0_'></a>    
- [Libraries](#toc1_)    
- [Loading text filings](#toc2_)    
- [Extracting the Management's Discussion and Analysis Section](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Libraries](#toc0_)

In [11]:
import os
import time
os.chdir(os.environ.get('PROJECT_PATH'))
from secnlp.ml_logic import data as d
from secnlp.ml_logic import parsing as p
import secnlp.ml_logic.parsing
from secnlp import utils as u
from secnlp.params import *
import pandas as pd
import importlib

# <a id='toc2_'></a>[Loading text filings](#toc0_)

In [12]:
df = u.read_data_from_bq(credentials = SERVICE_ACCOUNT, gcp_project = PROJECT, bq_dataset = DATASET_ID, table = FILINGS_10KQ_TABLE_ID)

In [13]:
df['date_filed'] = pd.to_datetime(df['date_filed'])

In [14]:
filing_sample_10k = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-K')].sample(1)
filing_sample_10k['raw_filing'] = filing_sample_10k['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


In [15]:
filing_sample_10q = df[(df['date_filed'].dt.year == 2023) & (df['form_type'] == '10-Q')].sample(1)
filing_sample_10q['raw_filing'] = filing_sample_10q['file_name'].apply(lambda url: d.fetch_text_from_url(url, agent = AGENT))


# 10-Ks

In [16]:
text_10k = filing_sample_10k['raw_filing'].iloc[0]

In [17]:
mda_10k = p.parse_10k_filing_items(text_10k, item = '7')

In [18]:
importlib.reload(secnlp.ml_logic.parsing)
p.cleaning(mda_10k)

'item 7 managements discussion and analysis of financial condition and results of operations in addition these statements constitute the companys cautionary statements under the private securities litigation reform act of 1995 it should be understood that it is not possible to predict or identify all such factors consequently the following should not be considered to be a complete discussion of all potential risks or uncertainties the words anticipate estimate expect project intend believe plan target forecast and similar expressions are intended to identify forwardlooking statements forwardlooking statements speak only as of the date of the document in which they are made the company disclaims any obligation or undertaking to provide any updates or revisions to any forwardlooking statement to reflect any change in the companys expectations or any change in events conditions or circumstances on which the forwardlooking statement is based it is advisable however to consult any further d

# 10-Q

In [19]:
text_10q = filing_sample_10q['raw_filing'].iloc[0]
mda_10q = p.parse_10q_filing_items(text_10q, item = '2')

In [20]:
importlib.reload(secnlp.ml_logic.parsing)
p.cleaning(mda_10q)

'item 2 managements discussion and analysis of financial condition and results of operations the following managements discussion and analysis of financial condition and results of operations provides information that management believes is relevant to an assessment and understanding of our consolidated results of operations and financial condition and changes in financial condition you should read the following discussion and analysis of financial condition and results of operations together with the accompanying unaudited condensed consolidated financial statements and the related notes to those statements of shapeways included in this report and the audited financial statements and notes thereto as of and for the year ended december 31 2022 which is contained in our annual report on form 10ka filed with the securities and exchange commission sec on august 4 2023 some of the information contained in this discussion and analysis or set forth elsewhere in this report including informat